In [1]:
#The magic starts here!
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#! pip install --upgrade seaborn

In [3]:
# IMPORT LIBS

In [4]:
import MDAnalysis
from MDAnalysis.analysis.dihedrals import Dihedral
import os as os
from numpy.linalg import norm
import math
import pandas as pd
#import seaborn as sns


In [5]:
# for output saving the trajectories. But.. maybe we do not even need to save them. 

#out_root_path='/'
#trajectories_out_path='%s/trajectoriesCOM_K13/'%out_root_path

#if not os.path.isdir(trajectories_out_path):
#    print ("Generating out dir", trajectories_out_path)
#    os.mkdir(trajectories_out_path)

In [6]:
# trajectories_path='/Users/fatima/Stanford/70S/Trajectories/ALL/Within15CompleteTunnel/cat_rename_trajs/'
# ref_struct_path='/Users/fatima/Stanford/70S/Trajectories/ALL/Within15CompleteTunnel/cat_rename_trajs/'
# out_dir='/Users/fatima/Stanford/WritingSecM/STACKING_R26/FRAMES/'

In [7]:
# run=0

# systdict={0:'K13', 1:'N13', 2:'WT'}

# traj=trajectories_path+"run%d.xtc" % run
# ref_struct=ref_struct_path+"top%d.pdb" % run
# u=MDAnalysis.Universe(ref_struct, traj)


In [8]:
def load_trajs(run, trajectories_path, ref_struct_path):
    
    traj=trajectories_path+"run%d.xtc" % run
    ref_struct=ref_struct_path+"top%d.pdb" % run
    u=MDAnalysis.Universe(ref_struct, traj)

    #Define side chains 
    G2505_SIDE= u.select_atoms("(name C3 or name N3 or name C4 or name C5 or name C6 or name N1) and (resname G and resid 2505)")
    ARG26_SIDE= u.select_atoms("(name NH1 or name NE or name NH2) and (resname ARG and resid 26)")
    
    U2585_SIDE= u.select_atoms("(name N1 or name C6 or name C5 or name C4 or name N3 or name C2) and (resname U and resid 2585)")
    ILE25_SIDE= u.select_atoms("(name CB or name CG1 or name CG2 or name CD) and (resname ILE and resid 25)")
 
    return(u,G2505_SIDE,ARG26_SIDE)


In [9]:
def calc_per100_stacking_his22(sidechain1, sidechain2):
    """Determine if stacking is present based on dihedrals and distances
    Input, 2 sidechains, loaded from MDAnalysis 
    This function is poorly written, and has hard-coded the dihedral angle 0_-
    Need to fix this function, to include traj information as well"""
    
    counter_p=0

    for ts in u.trajectory:
        #Let's first get the distance 
    
        A = sidechain1.center_of_mass()
        B = sidechain2.center_of_mass()
        dist=norm(A - B)
    
        #And let's get the angle in degrees
        #Note these are hard-coded 0_0 
    
        a1 = u.select_atoms("name N1 and resname U and resid 2609").positions
        a2 = u.select_atoms("name N3 and resname U and resid 2609").positions
        a3 = u.select_atoms("name C5 and resname U and resid 2609").positions
        a4 = u.select_atoms("name CG and resname HIS and resid 22").positions
        R=MDAnalysis.lib.distances.calc_dihedrals(a1, a2, a3, a4)
        degs=degrees(R)
    
        # Now, let's do the test 
    
        if degs <= 110 and degs >= 70:
            if dist<=5.0:
                counter_p +=1
    
    print("Stacking %: "+ str(counter_p/u.trajectory.n_frames))
    

    

In [10]:
def separate_frames_by_stacking_his22(sidechain1, sidechain2):
    """Determine if stacking is present based on dihedrals and distances
    Input, 2 sidechains, loaded from MDAnalysis 
    This function is poorly written, and has hard-coded the dihedral angle 0_-
    Need to fix this function, to include traj information as well"""
    
    counter_p=0
    framesitos=[]
    not_framesitos=[]
    
    for ts in u.trajectory:
        #Let's first get the distance 
    
        A = sidechain1.center_of_mass()
        B = sidechain2.center_of_mass()
        dist=norm(A - B)
    
        #And let's get the angle in degrees
        #Note these are hard-coded 0_0 
    
        a1 = u.select_atoms("name N1 and resname U and resid 2609").positions
        a2 = u.select_atoms("name N3 and resname U and resid 2609").positions
        a3 = u.select_atoms("name C5 and resname U and resid 2609").positions
        a4 = u.select_atoms("name CG and resname HIS and resid 22").positions
        R=MDAnalysis.lib.distances.calc_dihedrals(a1, a2, a3, a4)
        degs=degrees(R)
    
        # Now, let's do the test 
    
        if degs <= 110 and degs >= 70:
            if dist<=5.0:
                counter_p +=1
                framesitos.append(ts.frame)
                #print(ts.frame)
            else: 
                not_framesitos.append(ts.frame)
        else:
            not_framesitos.append(ts.frame)
            
                
    
    print("Stacking %: "+ str(counter_p/u.trajectory.n_frames))
    #print(framesitos)
    #print(not_framesitos)
    return framesitos, not_framesitos

In [11]:
def separate_frames_by_stacking_arg26(sidechain1, sidechain2):
    """Determine if stacking is present based on dihedrals and distances
    Input, 2 sidechains, loaded from MDAnalysis 
    This function is poorly written, and has hard-coded the dihedral angle 0_-
    Need to fix this function, to include traj information as well"""
    
    framesitos=[]
    not_framesitos=[]
    
    
    
    df = pd.DataFrame(columns = ['COM Dist', 'Dih Angle'])
    
    counter_p1=0
    counter_p2=0

    for ts in u.trajectory:
        #Let's first get the distance 
    
        A = sidechain1.center_of_mass()
        B = sidechain2.center_of_mass()
        dist=norm(A - B)
        
        a1 = u.select_atoms("name NH2 and resname ARG and resid 26").positions
        a2 = u.select_atoms("name NE and resname ARG and resid 26").positions
        a3 = u.select_atoms("name NH1 and resname ARG and resid 26").positions
        a4 = u.select_atoms("name C2 and resname G and resid 2505").positions
        R=MDAnalysis.lib.distances.calc_dihedrals(a1, a2, a3, a4)
        degs=degrees(R)
        
        df=df.append({'COM Dist' : dist, 'Dih Angle' : degs[0]}, ignore_index = True)
        
    
        if (degs <= 110 and degs >= 70) or (degs >= -110 and degs <= -70):
            if dist<=5.0:
                counter_p1 +=1
                framesitos.append(ts.frame)
            else:
                counter_p2 +=1
                not_framesitos.append(ts.frame)
        else:
            counter_p2 +=1
            not_framesitos.append(ts.frame)
                
        
    print("Stacking %: "+ str(100*((counter_p1)/(u.trajectory.n_frames))))
    
    return(framesitos, not_framesitos)
    

## Load Data and Run Calculating Functions

In [12]:
trajectories_path='/Users/fatima/Stanford/70S/Trajectories/ALL/Within15CompleteTunnel/cat_rename_trajs/'
ref_struct_path=trajectories_path
out_dir='/Users/fatima/Stanford/WritingSecM/STACKING_R26/FRAMES/'

systdict={0:'K13', 1:'N13', 2:'WT'}

In [13]:
run=1
u,G2505_SIDE,ARG26_SIDE=load_trajs(run, trajectories_path, ref_struct_path)

In [14]:
framesitos=separate_frames_by_stacking_arg26(ARG26_SIDE,G2505_SIDE)

Stacking %: 6.455592105263158


In [15]:
print("stacking: "+ str(len(framesitos[0])))
print("NOT stacking: "+ str(len(framesitos[1])))

stacking: 157
NOT stacking: 2275


In [16]:
#print(framesitos[0])

In [17]:
#u.write('c-alpha_some.dcd', frames=u.trajectory[[0,2,3]]) 

In [18]:
ag = u.select_atoms("all")

In [19]:
#ag.write(out_dir+systdict[run]+'_stacking_R26_2505.xtc', frames=u.trajectory[framesitos[0]])
#ag.write(out_dir+systdict[run]+'_NOTstacking_R26_2505.xtc', frames=u.trajectory[framesitos[1]])